<a href="https://colab.research.google.com/github/Kroojel/INT-Group-16/blob/main/INT2_Neural_Network.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

First we need to import the classes

In [ ]:
# Pytorch Imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision

# Data transformation imports
from torchvision import transforms

# General Libraries 
import matplotlib.pyplot as plt
import numpy as np

# Loss fucntion and optimiser imports
import torch.optim

Loading the dataset and the ten labels "classes" relating to the ten classifications

In [ ]:
batchSize = 128

# Taining set:

trainTransform = transforms.Compose(
    [
     transforms.ToTensor(),                                 # Turns image into tensor
     transforms.RandomAdjustSharpness(2, 0.5),              # 50% chance of increasing sharpness
     transforms.RandomAdjustSharpness(0, 0.5),              # 50% chance of increasing blur
     transforms.RandomAutocontrast(0.5),                    # 50% chance of increasing contrast
     transforms.RandomHorizontalFlip(0.5),                  # 50% chance of flipping the image horizontally
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalising the tensor
    ]
)

trainSet = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=trainTransform)

trainLoader = torch.utils.data.DataLoader(trainSet, batch_size=batchSize, shuffle=True, num_workers=2)

# Testing set:

testTransform = transforms.Compose(
    [
     transforms.ToTensor(),                                 # Turns image into tensor
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)) # Normalising the tensor
    ]
)

testSet = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=testTransform)

testLoader = torch.utils.data.DataLoader(testSet, batch_size=batchSize, shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


Defining neural network shape:

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn1 = nn.BatchNorm2d(12)
        self.conv2 = nn.Conv2d(in_channels=12, out_channels=12, kernel_size=5, stride=1, padding=1)
        self.bn2 = nn.BatchNorm2d(12)
        self.pool = nn.MaxPool2d(2,2)
        self.conv4 = nn.Conv2d(in_channels=12, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn4 = nn.BatchNorm2d(24)
        self.conv5 = nn.Conv2d(in_channels=24, out_channels=24, kernel_size=5, stride=1, padding=1)
        self.bn5 = nn.BatchNorm2d(24)
        self.fc1 = nn.Linear(2400, 50)
        self.fc2 = nn.Linear(50, 10)

    def forward(self, input):
        output = F.relu(self.bn1(self.conv1(input)))
        output = F.relu(self.bn2(self.conv2(output)))
        output = self.pool(output)
        output = F.relu(self.bn4(self.conv4(output)))
        output = F.relu(self.bn5(self.conv5(output)))
        output = output.view(-1, 2400)
        output = self.fc1(output)
        output = self.fc2(output)
        return output


net = Net()

Define optimiser and loss function